In [1]:
from gpt_generate import GPTgen
from datasets import load_dataset
import pandas as pd
from tqdm.auto import trange
import os

### Все гиперпараметры

In [2]:
model_path = 'ai-forever/rugpt3large_based_on_gpt2'
weights_path = 'weights/pref_with_word.pt'
dataset_path = 'shershen/ru_anglicism'
out_path = 'weights'
temp = 0.9
top_p = 0.7

### Класс для генерации

In [3]:
generator = GPTgen(model_path, weights_path)

### Тестовый датасет

In [4]:
df = load_dataset(dataset_path)
test_df = pd.DataFrame(df['test'])

No config specified, defaulting to: ru_anglicism/default
Found cached dataset ru_anglicism (/home/jovyan/.cache/huggingface/datasets/shershen___ru_anglicism/default/0.2.0/a1f71309ff0601f2a20ae30cd23f0b52822711d1af481586e36f525a21ed3bc2)


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
test_df.head(5)

,word,form,sentence,paraphrase
0,скринить,скринит,Эта дура скринит переписки и постит в инстаграм.,Эта дура фотографирует переписки и выставляет ...
1,вайб,вайб,"Не могу это объяснить, но у четверга и 16:00 о...","Не могу это объяснить, но у четверга и 16:00 о..."
2,вайб,вайб,"""Вчера выпал пушистый снег и в воздухе царил н...","""Вчера выпал пушистый снег и в воздухе царило ..."
3,чек-ап,чекап,Программа чекап в ИНВИТРО создана специально д...,Программа комплексного медицинского обследован...
4,чек-ап,Чек-Ап,"В ЛДЦ ""Кутузовский"" в Москве вы можете пройти ...","В ЛДЦ ""Кутузовский"" в Москве вы можете пройти ..."


### Пример генерации для одной пары англицизм/текст

In [6]:
texts = [test_df['form'].values[1], test_df['sentence'].values[1]]

In [7]:
texts

['вайб', 'Не могу это объяснить, но у четверга и 16:00 одинаковый вайб.']

In [8]:
paraphr = generator.generate(texts, temp=temp, top_p=top_p)

In [9]:
paraphr[0]

['Не могу это объяснить, но у четверга и 16:00 одинаковый вайб.']

### Генерация для всего тестового сета

In [10]:
def get_paraphrase(df, batch=16):
    paraphrs = []
    for pos in trange(0, len(df), batch):
        texts = [df['form'].values[pos:pos+batch].tolist(), df['sentence'].values[pos:pos+batch].tolist()]
        loc_paraphr = generator.generate(texts, temp=temp, top_p=top_p)
        paraphrs.extend(loc_paraphr[0])
    return paraphrs

In [11]:
all_p = get_paraphrase(test_df)

  0%|          | 0/5 [00:00<?, ?it/s]

In [12]:
test_copy = test_df.copy()
test_copy['preds'] = all_p

In [13]:
all_p[0:5]

['Эта дура пишет в инстаграм.',
 'Не могу это объяснить, но у четверга и 16:00 одинаковый набор будильников.',
 '"Вчера выпал пушистый снег и в воздухе царил настоящий рождественский снег".',
 'Программа обследования организма на ранней стадии создана специально для диагностики здоровья и выявления заболеваний на ранней стадии.',
 'В ЛДЦ "Кутузовский" в Москве вы можете пройти полное обследование всего организма.']

In [14]:
test_copy.to_csv(os.path.join(out_path, 'paraphr_w_p.csv'), index=False)